In [1]:
import pandas as pd
import numpy as np
import dimod
from itertools import combinations

%load_ext autoreload
%autoreload 1

%aimport modules
from modules import qubo_from_data
from modules import process_qubo_df
from modules import reduce_higher_order_terms
from modules import qubo_matrix_from_df
from modules import print_qubo_matrix
from modules import read_spot_file

## QUBO encoder

### Data description

Our task consists in optimizing the routine of a satellite that needs to take several photos. The satellite under consideration has three different cameras (1,2,3) and can take mono-photo, choosing one among the three cameras, or stereo-photo, with cameras 1 and 3.

The satellite's routine is scheduled by a .spot file that contains both photo requests and constraints on those. At the beginning of the requests, there is a single line stating the number of requests, the same for the constraints.

A photo request consists in:
- the id of the photo
- the value of the picture
- a list of different cameras with which the photo can be taken
- (after the camera id there is the memory used, not of our interest yet)

A constraint consists in:
- the number of requests affected by the constraint
- a list of camera combinations not allowed

Our goal is to maximize the value of each .spot schedule without breaking any constraints.


### Read spot file

In [191]:
# create empty photo requests dataframe
photo_req_df = pd.DataFrame({'id':      pd.Series(dtype='int'),
                             'value':   pd.Series(dtype='int'),
                             'mono':    pd.Series(dtype='bool'),
                             'options': pd.Series(dtype='object')})

In [192]:
# process photo requests
def request_from_line(line):
    # split line
    l = line.split()

    # get features
    id = int(l[0])
    value = int(l[1])
    mono = False if (l[2] == '1' and int(l[3])>4) else True
    options = [int(o) for o in l[3::2]]

    return list([id, value, mono, options])

In [193]:
# create empty constraints dataframe
constraints_df = pd.DataFrame({'ids':            pd.Series(dtype='object'),
                               'restrictions':   pd.Series(dtype='object')})

In [194]:
# process constraints
def costraint_from_line(line):
    # split line
    l = line.split()

    # get features
    n = int(l[0])
    ids = [int(i) for i in l[1:n+1]]
    restrictions = [[int(l[j]) for j in range(i,i+n)] for i in range(n+1, len(l), n)]

    return list([ids, restrictions])

In [195]:
# read test file
data_dir  = 'data/'
file_name = '15.spot'
with open(data_dir+file_name) as f:
    lines = f.readlines()

# get ids and reqs lengths
lens = [int(l.split()[0]) for l in lines if len(l.split())==1]

# create photo requests dataframe
for i in range(1,lens[0]+1):
    photo_req_df.loc[i-1] = request_from_line(lines[i])

# create constraints dataframe
for i in range(lens[0]+2, lens[0]+lens[1]+2):
    constraints_df.loc[i-(lens[0]+2)] = costraint_from_line(lines[i])

In [196]:
photo_req_df

,id,value,mono,options
0,0,1,True,"[1, 2, 3]"
1,1,2,False,[13]
2,2,1,True,"[1, 2, 3]"
3,3,1,True,"[1, 2, 3]"
4,4,2,True,[2]
5,5,2,False,[13]
6,6,2,False,[13]
7,7,2,False,[13]
8,8,1,True,"[1, 2, 3]"
9,9,1,True,"[1, 2, 3]"


In [197]:
constraints_df

,ids,restrictions
0,"[12, 3]","[[3, 3], [2, 2], [1, 1]]"
1,"[7, 2, 3]","[[13, 2, 3]]"
2,"[9, 7, 2]","[[3, 13, 2]]"
3,"[1, 10]","[[13, 13]]"
4,"[11, 7, 2]","[[3, 13, 2]]"
5,"[11, 3]","[[3, 3], [2, 2], [1, 1]]"
6,"[12, 11]","[[3, 3], [2, 2], [1, 1]]"
7,"[9, 3]","[[3, 3], [2, 2], [1, 1]]"
8,"[0, 3]","[[3, 3], [2, 2], [1, 1]]"
9,"[12, 0]","[[3, 3], [2, 2], [1, 1]]"


### Read spot file module

In [2]:
# read spot file
data_dir  = 'data/'
file_name = '20.spot'
photo_req_df, constraints_df = read_spot_file(data_dir, file_name)

In [31]:
photo_req_df.head()

,id,value,mono,options
0,0,1,True,"[1, 2, 3]"
1,1,1,True,"[1, 2, 3]"
2,2,1,True,"[1, 2, 3]"
3,3,1,True,"[1, 2, 3]"
4,4,2,False,[13]


In [32]:
constraints_df.head()

,ids,restrictions
0,"[1, 0]","[[3, 3], [2, 2], [1, 1]]"
1,"[2, 0]","[[3, 3], [2, 2], [1, 1]]"
2,"[3, 0]","[[3, 3], [2, 2], [1, 1]]"
3,"[5, 4]","[[13, 13]]"
4,"[5, 6]","[[13, 13]]"


### QUBO formulation
QUBO stays for Quadratic Uncostrained _Binary_ Optimization. First of all, we should translate our problem in terms of binary variables. We will consider two possibilities:

__standard__
- _mono_: We can take (1) or not take (0) the photo with each of the three cameras. Three binary variables are needed:
$$
(x_{i0},\, x_{i1},\, x_{i2})
$$
- _stereo_: The stereo photo can only be taken with the cameras $1$ and $3$, so we have only one binary variable:
$$
x_i
$$

Note that, in the case of a mono photo, we have a total of 8 different instantiations of the binary variables when only 3 are feasible ($001,\, 010,\, 100$). We can think about more efficient encodings.

__dense__:
- _mono_: Just two variables could be sufficient:

<center>

|          | $x_{i0}$ | $x_{i1}$ |
|----------|----------|----------|
| no photo | 0        | 0        |
| camera 1 | 0        | 1        |
| camera 2 | 1        | 0        |
| camera 3 | 1        | 1        |

</center>

- _stereo_: As in the previous encoding we consider one variable
$$
x_i
$$

In the latter case we use one less variable with respect to the former, it seems convenient. Despite that, with the _dense_ encoding the costraint formulation requires an auxiliary variable so it is important to carefully understand if there is an actual advantage.

#### Standard encoding

In [24]:
# create empty qubo dataframe
qubo_df = pd.DataFrame({'rank':    pd.Series(dtype='int'),
                        'coeff':   pd.Series(dtype='int'),
                        'indexes': pd.Series(dtype='object')})

In [25]:
# formulate qubo istance from photo request
def qubo_from_request(request, option):
    # get features
    id = request[0]
    value = request[1]
    mono = request[2]
    camera = request[3][option]

    # get qubo
    rank = 1
    coeff = -value # minimize qubo -> maximize value
    indexes = [[id, camera]] # if mono else [id]
    qubo = list([rank, coeff, indexes])

    return qubo

In [26]:
 # formulate qubo instance from constraint
def qubo_from_constraint(constraint, option, coeff):
    # get features
    ids = constraint[0]
    restriction = constraint[1][option]

    # get qubo
    rank = len(ids)
    coeff = coeff
    indexes = [[id, camera] for id, camera in zip(ids, restriction)]
    qubo = list([rank, coeff, indexes])

    return qubo

In [27]:
k = 0

# populate qubo dataframe from photo requests
for i in range(len(photo_req_df)):
    l = len(photo_req_df.loc[i]['options'])
    for j in range(l):
        qubo_df.loc[k+j] = qubo_from_request(photo_req_df.loc[i], j)
    k = k + l

# penalties coefficient
m = -1.1*min(qubo_df['coeff'])                                                                           

# add penalties to avoid taking the same photo multiple times with different cameras
for i in range(len(photo_req_df)): 
    if len(photo_req_df.loc[i]['options'])>1 :
        for j, z in combinations(photo_req_df.loc[i]['options'], 2):
            qubo_df.loc[k] = list([2, m, [[i, photo_req_df.loc[i]['options'][j-1]], [i, photo_req_df.loc[i]['options'][z-1]]]])
            k = k + 1

# populate qubo dataframe from constraints
for i in range(len(constraints_df)):
    l = len(constraints_df.loc[i]['restrictions'])
    for j in range(l):
        qubo_df.loc[k+j] = qubo_from_constraint(constraints_df.loc[i], j, m)
    k = k + l

In [28]:
qubo_df

,rank,coeff,indexes
0,1,-1.0,"[[0, 1]]"
1,1,-1.0,"[[0, 2]]"
2,1,-1.0,"[[0, 3]]"
3,1,-1.0,"[[1, 1]]"
4,1,-1.0,"[[1, 2]]"
5,1,-1.0,"[[1, 3]]"
6,1,-1.0,"[[2, 1]]"
7,1,-1.0,"[[2, 2]]"
8,1,-1.0,"[[2, 3]]"
9,1,-1.0,"[[3, 1]]"


In [31]:
"""# group qubo dataframe by indexes TODO: check if it is necessary ( it seems not )
def preprocess_indexes(indexes, join=False):
    # sort if more than one element
    if len(indexes)>1:
        indexes = sorted(indexes, key=lambda x: x[0]+x[1]/10) # hardcoded solution to consider both sublists elements

    # produce string output
    proc_indexes = [str(i[0])+str(i[1]) for i in indexes ]
    return ''.join(proc_indexes) if join else proc_indexes

# group by operation needs string object
qubo_df['gb_indexes'] = qubo_df['indexes'].apply(preprocess_indexes, join=True)
qubo_df_ = qubo_df.groupby(['gb_indexes'], as_index=False, sort=False).agg({'rank': 'first', 'coeff': 'sum', 'indexes': 'first'})#.drop(['gb_indexes'], axis=1)
# recover process indexes for qubo indexing
#qubo_df_['proc_indexes'] = qubo_df_['indexes'].apply(preprocess_indexes, join=False)"""
def preprocess_indexes(indexes, join=False):
    # sort if more than one element
    if len(indexes)>1:
        indexes = sorted(indexes, key=lambda x: x[0]+x[1]/10) # hardcoded solution to consider both sublists elements

    # produce string output
    proc_indexes = [str(i[0])+str(i[1]) for i in indexes ]
    return ''.join(proc_indexes) if join else proc_indexes
    
# produce hash column for dictionary indexing
qubo_df['keys'] = qubo_df['indexes'].apply(preprocess_indexes, join=False)

In [32]:
# produce 1 to one dict between keys of the dataframe and indexes of the qubo matrix
key_to_qubo_dict = {}
for i in range(len(qubo_df)):
    # check if index is a single key
    if len(qubo_df.loc[i]['keys'])==1:
        key_to_qubo_dict[qubo_df.loc[i]['keys'][0]] = i

In [33]:
# populate qubo matrix dictionary
qubo = {}
for i in range(len(qubo_df)):
    # check if index is a single key
    keys = qubo_df.loc[i]['keys']
    if len(keys)==1:
        idx = key_to_qubo_dict[keys[0]]
        # check if index is already in dictionary
        qubo[(idx, idx)] = qubo[(idx, idx)]+qubo_df.loc[i]['coeff'] if (idx, idx) in qubo else qubo_df.loc[i]['coeff']
    else:
        # assume quadratic term, higher order will be treated before 
        idx_0 = key_to_qubo_dict[keys[0]]
        idx_1 = key_to_qubo_dict[keys[1]]
        # check if index is already in dictionary
        qubo[(idx_0, idx_1)] = qubo[(idx_0, idx_1)]+qubo_df.loc[i]['coeff'] if (idx_0, idx_1) in qubo else qubo_df.loc[i]['coeff']

In [34]:
# print qubo matrix
def print_qubo(qubo, key_to_qubo_dict):
    for i in range(len(key_to_qubo_dict)):
        for j in range(len(key_to_qubo_dict)):
            print(qubo[(i, j)], end=' ') if (i,j) in qubo else print(0, end=' ')
        print()

print_qubo(qubo, key_to_qubo_dict)

-1.0 2.2 2.2 2.2 0 0 2.2 0 0 2.2 0 0 0 0 0 0 
0 -1.0 2.2 0 2.2 0 0 2.2 0 0 2.2 0 0 0 0 0 
0 0 -1.0 0 0 2.2 0 0 2.2 0 0 2.2 0 0 0 0 
0 0 0 -1.0 2.2 2.2 2.2 0 0 2.2 0 0 0 0 0 0 
0 0 0 0 -1.0 2.2 0 2.2 0 0 2.2 0 0 0 0 0 
0 0 0 0 0 -1.0 0 0 2.2 0 0 2.2 0 0 0 0 
0 0 0 0 0 0 -1.0 2.2 2.2 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 -1.0 2.2 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 -1.0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 -1.0 2.2 2.2 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 -1.0 2.2 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 -1.0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 -2.0 2.2 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 -2.0 2.2 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 -2.0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -2.0 


#### Dense encoding

In [11]:
# create empty qubo dataframe
qubo_df = pd.DataFrame({'rank':    pd.Series(dtype='int'),
                        'coeff':   pd.Series(dtype='int'),
                        'indexes': pd.Series(dtype='object')})

In [12]:
dense_encoding_dict = {1: [0, 1], 2: [1, 0], 3: [1, 1], 13: [13]}

In [13]:
# produce qubo indexing from single camera request
def index_from_camera(camera, id):
    # retrive encoding information
    enc = dense_encoding_dict[camera]
    # stereo case
    if len(enc)==1:
        return [(1, [[id, enc[0]]])]
    # mono case
    out = []
    if enc[0] and enc[1] :
        out.append((1, [[id, 0], [id, 1]]))
    else:
        for j in range(2):
            if enc[j]:
                out.append((1, [[id, j]]))
            else:
                out.append((-1, [[id, 0], [id, 1]]))
    return out

In [14]:
# formulate qubo from photo request
def qubo_from_request(request, option):
    # get features
    id = request[0]
    value = request[1]
    mono = request[2]
    camera = request[3][option]

    # get indexing
    idx = index_from_camera(camera, id)

    # get qubos
    qubo = []
    for i in idx:
        rank = len(i[1])
        coeff = -value*i[0]
        indexes = i[1]
        qubo.append(list([rank, coeff, indexes]))

    return qubo

In [15]:
 # formulate qubo instance from constraint
def qubo_from_constraint(constraint, option, coeff):
    # get features
    ids = constraint[0]
    restriction = constraint[1][option]

    #print("ids: ",ids,"restriction: ",restriction)

    # compress idxs to a single idx
    idxs = []
    for i in range(len(restriction)):
        idxs.append(index_from_camera(restriction[i], ids[i]))
    ## number of resulting qubo terms
    while len(idxs)!=1:
        idxs_ = []
        print(idxs)
        for i in idxs[0]:
            print(i)
            for j in idxs[1]:
                print(j)
                idxs_.append((i[0]*j[0], i[1]+j[1]))
                print(idxs_)
        idxs = [idxs_] + idxs[2:]
        print(idxs)

    idx = idxs[0]
    
    # get qubo terms
    qubos = []    
    for i in idx:
        rank = len(i[1])
        print(i[0])
        coeff_ = coeff*i[0]
        indexes = i[1]
        qubos.append(list([rank, coeff_, indexes]))
    
    return qubos

In [18]:
qubo_from_constraint([[10, 2], [[13, 1]]], 0, 1)

[[(1, [[10, 13]])], [(-1, [[2, 0], [2, 1]]), (1, [[2, 1]])]]
(1, [[10, 13]])
(-1, [[2, 0], [2, 1]])
[(-1, [[10, 13], [2, 0], [2, 1]])]
(1, [[2, 1]])
[(-1, [[10, 13], [2, 0], [2, 1]]), (1, [[10, 13], [2, 1]])]
[[(-1, [[10, 13], [2, 0], [2, 1]]), (1, [[10, 13], [2, 1]])]]
-1
1


[[3, -1, [[10, 13], [2, 0], [2, 1]]], [2, 1, [[10, 13], [2, 1]]]]

In [143]:
k = 0

# populate qubo dataframe from photo requests
for i in range(len(photo_req_df)):
    l = len(photo_req_df.loc[i]['options'])
    for j in range(l):
        qubo_inst = qubo_from_request(photo_req_df.loc[i], j)
        for q in qubo_inst:
            qubo_df.loc[k] = q
            k = k + 1

# penalties coefficient
m = -1.1*min(qubo_df['coeff'])                                                                           

# populate qubo dataframe from constraints
for i in range(len(constraints_df)):
    l = len(constraints_df.loc[i]['restrictions'])
    for j in range(l):
        qubo_inst = qubo_from_constraint(constraints_df.loc[i], j, m)
        for q in qubo_inst:
            qubo_df.loc[k] = q
            k = k + 1

In [150]:
def preprocess_indexes(indexes, join=False):
    # sort if more than one element
    if len(indexes)>1:
        indexes = sorted(indexes, key=lambda x: x[0]+x[1]/10) # hardcoded solution to consider both sublists elements

    # produce string output
    proc_indexes = [str(i[0])+str(i[1]) for i in indexes ]
    return ''.join(proc_indexes) if join else proc_indexes
    
# produce hash column for dictionary indexing
qubo_df['keys'] = qubo_df['indexes'].apply(preprocess_indexes, join=False)

In [151]:
# produce 1 to one dict between keys of the dataframe and indexes of the qubo matrix
key_to_qubo_dict = {}
for i in range(len(qubo_df)):
    # check if index is a single key
    if len(qubo_df.loc[i]['keys'])==1:
        key_to_qubo_dict[qubo_df.loc[i]['keys'][0]] = i

In [152]:
key_to_qubo_dict

{'01': 1,
 '00': 2,
 '11': 6,
 '10': 7,
 '21': 11,
 '20': 12,
 '31': 16,
 '30': 17,
 '413': 20,
 '513': 21,
 '613': 22,
 '713': 23}

### Encoding module

In [21]:
# select encoding (standard, dense)
encoding = 'dense'
penalty_coeff = 1.1

# get qubo df
qubo_df = qubo_from_data(photo_req_df, constraints_df, encoding, penalty_coeff)

In [25]:
qubo_df.head()

,rank,coeff,indexes
0,1,-2.0,"[[0, 13]]"
1,2,1.0,"[[1, 0], [1, 1]]"
2,1,-1.0,"[[1, 1]]"
3,1,-1.0,"[[1, 0]]"
4,2,1.0,"[[1, 0], [1, 1]]"


### Preprocessing

In [234]:
# group terms with same indexing and sum coefficients
def preprocess_indexes(indexes, join=False):
    # sort if more than one element
    if len(indexes)>1:
        indexes = sorted(indexes, key=lambda x: x[0]+x[1]/10) # hardcoded solution to consider both sublists elements

    # produce string output
    proc_indexes = [str(i[0])+'_'+str(i[1]) for i in indexes ]
    return ''.join(proc_indexes) if join else proc_indexes

# group by operation needs string object
qubo_df['gb_indexes'] = qubo_df['indexes'].apply(preprocess_indexes, join=True)
qubo_df_ = qubo_df.groupby(['gb_indexes'], as_index=False, sort=False).agg({'rank': 'first', 'coeff': 'sum', 'indexes': 'first'}).drop(['gb_indexes'], axis=1)

# recover process indexes for qubo indexing
#qubo_df_['proc_indexes'] = qubo_df_['indexes'].apply(preprocess_indexes, join=False)

In [235]:
# produce hash column for dictionary indexing
qubo_df_['keys'] = qubo_df_['indexes'].apply(preprocess_indexes, join=False)

In [236]:
qubo_df_

,rank,coeff,indexes,keys
0,1,-1.0,"[[0, 1]]",[0_1]
1,1,-1.0,"[[0, 2]]",[0_2]
2,1,-1.0,"[[0, 3]]",[0_3]
3,1,-2.0,"[[1, 13]]",[1_13]
4,1,-1.0,"[[2, 1]]",[2_1]
...,...,...,...,...
82,2,2.2,"[[0, 1], [11, 1]]","[0_1, 11_1]"
83,2,2.2,"[[4, 2], [13, 2]]","[4_2, 13_2]"
84,2,2.2,"[[12, 3], [9, 3]]","[9_3, 12_3]"
85,2,2.2,"[[12, 2], [9, 2]]","[9_2, 12_2]"


In [237]:
# produce 1 to one dict between keys of the dataframe and indexes of the qubo matrix
keys = np.array([k_i for key in qubo_df_['keys'] for k_i in key])
keys = np.unique(keys)
key_to_qubo_dict = { key:i for i,key in enumerate(keys) }

In [238]:
key_to_qubo_dict

{'0_1': 0,
 '0_2': 1,
 '0_3': 2,
 '10_13': 3,
 '11_1': 4,
 '11_2': 5,
 '11_3': 6,
 '12_1': 7,
 '12_2': 8,
 '12_3': 9,
 '13_1': 10,
 '13_2': 11,
 '13_3': 12,
 '14_13': 13,
 '1_13': 14,
 '2_1': 15,
 '2_2': 16,
 '2_3': 17,
 '3_1': 18,
 '3_2': 19,
 '3_3': 20,
 '4_2': 21,
 '5_13': 22,
 '6_13': 23,
 '7_13': 24,
 '8_1': 25,
 '8_2': 26,
 '8_3': 27,
 '9_1': 28,
 '9_2': 29,
 '9_3': 30}

In [239]:
# add column with variable indexing
qubo_df_['variables'] = qubo_df_['keys'].apply(lambda k: [key_to_qubo_dict[k_i] for k_i in k])
qubo_df_ = qubo_df_.drop(['indexes', 'keys'], axis=1)
qubo_df_ ['variables'] = qubo_df_ ['variables'].apply(lambda x: np.array(x))

In [240]:
qubo_df_

,rank,coeff,variables
0,1,-1.0,[0]
1,1,-1.0,[1]
2,1,-1.0,[2]
3,1,-2.0,[14]
4,1,-1.0,[15]
...,...,...,...
82,2,2.2,"[0, 4]"
83,2,2.2,"[21, 11]"
84,2,2.2,"[30, 9]"
85,2,2.2,"[29, 8]"


In [207]:
print("Total number of variables: ",len(key_to_qubo_dict))

Total number of variables:  31


### Preprocessing module

In [50]:
qubo_df_, key_to_qubo_dict = process_qubo_df(qubo_df)

In [51]:
qubo_df_.head()

,rank,coeff,variables
0,1,-2.0,[0]
1,2,1.0,"[14, 15]"
2,1,-1.0,[15]
3,1,-1.0,[14]
4,2,1.0,"[16, 17]"


In [282]:
print(key_to_qubo_dict)

{'0_1': 0, '0_2': 1, '0_3': 2, '10_13': 3, '11_1': 4, '11_2': 5, '11_3': 6, '12_1': 7, '12_2': 8, '12_3': 9, '13_1': 10, '13_2': 11, '13_3': 12, '14_13': 13, '1_13': 14, '2_1': 15, '2_2': 16, '2_3': 17, '3_1': 18, '3_2': 19, '3_3': 20, '4_2': 21, '5_13': 22, '6_13': 23, '7_13': 24, '8_1': 25, '8_2': 26, '8_3': 27, '9_1': 28, '9_2': 29, '9_3': 30}


### Higher order terms

#### Boros

In [28]:
## Boros algorithm to reduce higher order terms in quadratic ones

# set M = 1 + sum(|c_i|), m = n
M = sum(abs(qubo_df_['coeff']))+1
#print(M)
m = len(key_to_qubo_dict)
#print(m)

# while there exist a term with rank > 2 
while max(qubo_df_['rank'])>2:
    # qubo to key dictionary
    qubo_to_key_dict = {v: k for k, v in key_to_qubo_dict.items()}

    qubo = []
    # select an higher order term
    ho_term = np.array(qubo_df_[qubo_df_['rank']>2]['variables'])[0]
    #print(ho_term)

    # choose two elements from it 
    ij = ho_term[:2]
    #print(ij)

    # select terms with the two elements
    mask = [all(x in var for x in ij) for var in qubo_df_['variables']]
    ho_df = qubo_df_[mask]
    #print(ho_df)

    # update key_to_qubo_dict with the new variable
    key_to_qubo_dict['_'.join([qubo_to_key_dict[v] for v in ij])] = m

    # update i,j term
    i = ho_df.index[ho_df['rank']==2].to_list()
    if len(i)!=0:
        #print("found bin term", i[0])
        #qubo_df_.loc[i[0], 'coeff'] = qubo_df_.loc[i[0], 'coeff'] + M
        #qubo_df_.loc[i[0], 'variables'] = np.array([m])
        qubo.append(list([2, qubo_df_.loc[i[0], 'coeff'] + M, np.array([v for v in ij])]))
        qubo_df_.drop(i[0], inplace=True)
        ho_df = ho_df.drop(i[0])
    else:
        qubo.append(list([2, M, np.array([v for v in ij])]))

    # create terms [i, m; j, m]
    qubo.append(list([2, -2*M, np.array([ij[0], m])]))
    qubo.append(list([2, -2*M, np.array([ij[1], m])]))

    # create term m
    qubo.append(list([1, 3*M, np.array([m])]))
    #print(qubo)

    # change variables where i,j appear to m
    for i in ho_df.index:
        #print(i)
        mask_ = np.array([var not in ij for var in qubo_df_.loc[i, 'variables']])
        #print(mask_)
        #print(qubo_df_.loc[i, "variables"])
        var = np.append(qubo_df_.loc[i, "variables"][mask_],[m])
        qubo.append(list([len(var), qubo_df_.loc[i, 'coeff'], var]))
        #qubo_df_.loc[i, 'variables'] = 
        #qubo_df_.loc[i, 'rank'] = len(qubo_df_.loc[i, 'variables'])
        qubo_df_.drop(i, inplace=True)

    # add new terms to qubo_df_
    #print(qubo)
    qubo_df_.reset_index(drop=True, inplace=True)
    #print(qubo_df_)
    l = len(qubo_df_)
    for i in range(len(qubo)):
        qubo_df_.loc[i+l] = qubo[i]

    # update m
    m = m + 1

In [62]:
qubo_df_

,rank,coeff,variables
0,1,-1.0,[1]
1,1,-1.0,[3]
2,1,-1.0,[2]
3,1,-1.0,[5]
4,1,-1.0,[4]
...,...,...,...
57,2,27.6,"[6, 9]"
58,2,-55.2,"[6, 16]"
59,2,-55.2,"[9, 16]"
60,1,82.8,[16]


In [63]:
# update qubo to key dictionary
qubo_to_key_dict = {v: k for k, v in key_to_qubo_dict.items()}

In [64]:
key_to_qubo_dict

{'0_0': 0,
 '0_1': 1,
 '1_0': 2,
 '1_1': 3,
 '2_0': 4,
 '2_1': 5,
 '0_0_0_1': 6,
 '0_1_1_0': 7,
 '0_1_1_1': 8,
 '1_0_1_1': 9,
 '1_1_2_0': 10,
 '1_0_2_0': 11,
 '1_1_2_1': 12,
 '2_0_2_1': 13,
 '2_1_0_0_0_1': 14,
 '2_1_0_1_1_0': 15,
 '0_0_0_1_1_0_1_1': 16}

#### Ishikawa

In [ ]:
qubo_df_[qubo_df_['rank']>2]

In [26]:
## Ishikawa algorithm to reduce higher order terms in quadratic ones

# current number of variables
w = len(key_to_qubo_dict)

# select higher order terms
ho_terms = qubo_df_[qubo_df_['rank']>2]
print(len(ho_terms), "higher order terms found")

# list for new terms
qubo = []

# reduce higher order terms
for i in ho_terms.index:
    # coefficient of the higher order term
    a = qubo_df_.loc[i, 'coeff']
    # order of the higher order term
    d = qubo_df_.loc[i, 'rank']
    # distinguish a<0 and a>0
    if a<0:
        # create new ancillary variable
        key_to_qubo_dict[f'{i}_ho_term'] = w
        # append new terms
        for var in qubo_df_.loc[i, 'variables']:
            qubo.append(list([2, a, np.array([w, var])]))
        qubo.append(list([1, -(d-1)*a, np.array([w])]))
        # delete ho_term
        qubo_df_.drop(i, inplace=True)
        # update w
        w = w + 1
    else:
        nd = int(np.floor((d-1)/2))
        for j in range(1, nd+1):
            c_jd = 1 if (j==nd and d%2!=0) else 2
            # create new ancillary variable
            key_to_qubo_dict[f'{i}_ho_term_{j}'] = w
            # append new terms
            for var in qubo_df_.loc[i, 'variables']:
                qubo.append(list([2, -a*c_jd, np.array([w, var])]))
            qubo.append(list([1, a*(2*j*c_jd-1), np.array([w])]))
            w = w + 1
        for j in range(d-1):
            for k in range(j+1, d):
                var_j = qubo_df_.loc[i, 'variables'][j]
                var_k = qubo_df_.loc[i, 'variables'][k]
                qubo.append(list([2, a, np.array([var_j, var_k])]))
        # delete ho_term
        qubo_df_.drop(i, inplace=True)

# add new terms to qubo_df_
#print(qubo)
qubo_df_.reset_index(drop=True, inplace=True)
#print(qubo_df_)
l = len(qubo_df_)
for i in range(len(qubo)):
    qubo_df_.loc[i+l] = qubo[i]

54 higher order terms found


In [243]:
# update qubo to key dictionary
qubo_to_key_dict = {v: k for k, v in key_to_qubo_dict.items()}

In [246]:
# group terms with same indexing and sum coefficients
def preprocess_variables(variables, join=False):
    # sort if more than one element
    if len(variables)>1:
        variables = sorted(variables) # hardcoded solution to consider both sublists elements

    # produce string output
    proc_variables = [str(i) for i in variables ]
    return '_'.join(proc_variables) if join else proc_variables

In [247]:
# group by operation needs string object
qubo_df_['gb_variables'] = qubo_df_['variables'].apply(preprocess_variables, join=True)

In [248]:
# gruop by qubo_df_ by gb_variables summing coefficients
qubo_df_ = qubo_df_.groupby(['gb_variables'], as_index=False, sort=False).agg({'rank':'first', 'coeff': 'sum', 'variables':'first'})

In [ ]:
qubo_df_.drop(['gb_variables'], axis=1, inplace=True)

#### Mixing Boros and Ishikawa

In [10]:
key_to_qubo_dict

{'0_13': 0,
 '10_13': 1,
 '11_0': 2,
 '11_1': 3,
 '12_13': 4,
 '13_13': 5,
 '14_13': 6,
 '15_0': 7,
 '15_1': 8,
 '16_0': 9,
 '16_1': 10,
 '17_13': 11,
 '18_13': 12,
 '19_13': 13,
 '1_0': 14,
 '1_1': 15,
 '2_0': 16,
 '2_1': 17,
 '3_13': 18,
 '4_13': 19,
 '5_0': 20,
 '5_1': 21,
 '6_13': 22,
 '7_0': 23,
 '7_1': 24,
 '8_0': 25,
 '8_1': 26,
 '9_13': 27}

In [9]:
qubo_df_[qubo_df_['rank']>2].head(20)

,rank,coeff,variables
36,4,6.6,"[14, 15, 16, 17]"
38,3,-2.2,"[14, 15, 16]"
39,3,-2.2,"[14, 16, 17]"
40,3,-2.2,"[15, 16, 17]"
41,3,-2.2,"[14, 15, 17]"
43,3,0.0,"[16, 17, 1]"
46,3,0.0,"[14, 15, 1]"
50,4,6.6,"[16, 17, 9, 10]"
52,3,-2.2,"[16, 9, 10]"
53,3,-2.2,"[16, 17, 9]"


In [6]:
# group terms with same indexing and sum coefficients
def preprocess_variables(variables, join=False):
    # sort if more than one element
    if len(variables)>1:
        variables = sorted(variables) # hardcoded solution to consider both sublists elements

    # produce string output
    proc_variables = [str(i) for i in variables ]
    return '_'.join(proc_variables) if join else proc_variables

In [53]:
m = len(key_to_qubo_dict)

# select higher order terms
ho_terms = qubo_df_[qubo_df_['rank']>2]
print(len(ho_terms), "higher order terms found")

# set M = 1 + sum(|c_i|)
M = sum(abs(qubo_df_['coeff']))+1

qubo_to_key_dict = {v: k for k, v in key_to_qubo_dict.items()}

# reduce higher order terms
while len(ho_terms)>0:#ho_terms.index:
    i = ho_terms.index[0]
    print("higher order term to be reduced\n", list(qubo_df_.iloc[i]))
    # coefficient of the higher order term
    a = qubo_df_.loc[i, 'coeff']
    # order of the higher order term
    d = qubo_df_.loc[i, 'rank']

    print("new term, last variable", list(key_to_qubo_dict.values())[-1])
    # distinguish a<0 and a>0

    if a<0: # use ishikawa

        print("using ishikawa")

        # list for new terms
        qubo = []
        # create new ancillary variable
        #print("new variable", m)
        key_to_qubo_dict[f'{m}_ho_term'] = m
        # qubo to key dictionary
        qubo_to_key_dict = {v: k for k, v in key_to_qubo_dict.items()}
        # append new terms
        for var in qubo_df_.loc[i, 'variables']:
            qubo.append(list([2, a, np.array([m, var])]))
        qubo.append(list([1, -(d-1)*a, np.array([m])]))
        # delete ho_term
        qubo_df_.drop(i, inplace=True)
        # update m
        m = m + 1

        # add new terms to qubo_df_
        qubo_df_.reset_index(drop=True, inplace=True)
        l = len(qubo_df_)
        for i in range(len(qubo)):
            qubo_df_.loc[i+l] = qubo[i]

        # group by operation needs string object
        qubo_df_['gb_variables'] = qubo_df_['variables'].apply(preprocess_variables, join=True)
        # gruop by qubo_df_ by gb_variables summing coefficients
        qubo_df_ = qubo_df_.groupby(['gb_variables'], as_index=False, sort=False).agg({'rank':'first', 'coeff': 'sum', 'variables':'first'})
        qubo_df_.drop(['gb_variables'], axis=1, inplace=True)
        
    if a>0: # use boros

        print("using boros")

        ho_term = np.array(qubo_df_.loc[i, 'variables'])

        old_m = m
        
        for j in range(d-2):
            # list for new terms
            qubo = []
            # sort to better handle picking pairs
            ho_term.sort()
            # choose two elements from it 
            ho_pair = ho_term[:2]
            ho_term = np.delete(ho_term, [0,1])

            # select terms with the two elements
            mask = [all(x in var for x in ho_pair) for var in qubo_df_['variables']]
            ho_df = qubo_df_[mask]

            # update key_to_qubo_dict with the new variable
            #print("new variable", m)
            key_to_qubo_dict['_'.join([qubo_to_key_dict[v] for v in ho_pair])] = m
            # qubo to key dictionary
            qubo_to_key_dict = {v: k for k, v in key_to_qubo_dict.items()}
            # update ho_term with the new variable
            ho_term = np.append(ho_term, m)

            # update ho_pair term
            pair_idx = ho_df.index[ho_df['rank']==2].to_list()
            if len(pair_idx)!=0:
                qubo.append(list([2, qubo_df_.loc[pair_idx[0], 'coeff'] + M, np.array([v for v in ho_pair])]))
                qubo_df_.drop(pair_idx[0], inplace=True)
                ho_df = ho_df.drop(pair_idx[0])
            else:
                qubo.append(list([2, M, np.array([v for v in ho_pair])]))

            # create terms [ho_pair[0], m; ho_pair[1], m]
            qubo.append(list([2, -2*M, np.array([ho_pair[0], m])]))
            qubo.append(list([2, -2*M, np.array([ho_pair[1], m])]))

            # create term m
            qubo.append(list([1, 3*M, np.array([m])]))

            # change variables where ho_pair appear to m
            for indexes in ho_df.index:
                mask_ = np.array([var not in ho_pair for var in qubo_df_.loc[indexes, 'variables']])
                var = np.append(qubo_df_.loc[indexes, "variables"][mask_],[m])
                qubo.append(list([len(var), qubo_df_.loc[indexes, 'coeff'], var]))
                qubo_df_.drop(indexes, inplace=True)

            # update m
            m = m + 1

            # add new terms to qubo_df_
            qubo_df_.reset_index(drop=True, inplace=True)
            l = len(qubo_df_)
            for i in range(len(qubo)):
                qubo_df_.loc[i+l] = qubo[i]

        #print(f'{m-old_m} new variables created')

    ho_terms = qubo_df_[qubo_df_['rank']>2]   

56 higher order terms found
higher order term to be reduced
 [4, 6.6000000000000005, array([14, 15, 16, 17])]
new term, last variable 27
using boros
higher order term to be reduced
 [3, -2.2, array([16,  9, 10])]
new term, last variable 29
using ishikawa
higher order term to be reduced
 [3, -2.2, array([17,  9, 10])]
new term, last variable 30
using ishikawa
higher order term to be reduced
 [3, -2.2, array([14,  9, 10])]
new term, last variable 31
using ishikawa
higher order term to be reduced
 [3, -2.2, array([15,  9, 10])]
new term, last variable 32
using ishikawa
higher order term to be reduced
 [3, -2.2, array([16, 23, 24])]
new term, last variable 33
using ishikawa
higher order term to be reduced
 [3, -2.2, array([17, 23, 24])]
new term, last variable 34
using ishikawa
higher order term to be reduced
 [3, -2.2, array([14, 23, 24])]
new term, last variable 35
using ishikawa
higher order term to be reduced
 [3, -2.2, array([15, 23, 24])]
new term, last variable 36
using ishikawa
hig

In [113]:
qubo_df_

,rank,coeff,variables
0,1,-1.0,[1]
1,1,-1.0,[0]
2,1,-2.0,[10]
3,2,1.0,"[11, 12]"
4,1,-1.0,[12]
...,...,...,...
171,1,4.4,[45]
172,2,-2.2,"[46, 12]"
173,2,-2.2,"[46, 26]"
174,2,-2.2,"[46, 39]"


#### Module

In [132]:
# set method (boros, ishikawa)
method = 'mix'

# reduce higher order terms
qubo_df_, key_to_qubo_dict = reduce_higher_order_terms(qubo_df_, key_to_qubo_dict, method)

KeyboardInterrupt: 

In [122]:
qubo_df_.head()

,rank,coeff,variables
0,1,-1.0,[1]
1,1,-1.0,[0]
2,1,-2.0,[10]
3,2,1.0,"[11, 12]"
4,1,-1.0,[12]


### QUBO matrix

In [123]:
qubo = qubo_matrix_from_df(qubo_df_)

In [13]:
print_qubo_matrix(qubo, key_to_qubo_dict)

-1.0 2.2 2.2 0.00 2.2 0.00 0.00 2.2 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 2.2 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 -1.0 2.2 0.00 0.00 2.2 0.00 0.00 2.2 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 2.2 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 -1.0 0.00 0.00 0.00 2.2 0.00 0.00 2.2 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 2.2 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 -2.0 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 0.00 -1.0 2.2 2.2 2.2 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 0.00 0.00 -1.0 2.2 0.00 2.2 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 0.00 0

### Exact solver

In [14]:
# encode the dataframe into a qubo object
bqm = dimod.BQM.from_qubo(qubo)
# exact solver
sampler_exact = dimod.ExactSolver()
sampleset = sampler_exact.sample(bqm)

MemoryError: Unable to allocate 264. GiB for an array with shape (8589934592, 33) and data type int8

In [182]:
# save results in a dataframe
results_df = sampleset.to_pandas_dataframe()
# recover original indexes
qubo_to_key_dict = {v: k for k, v in key_to_qubo_dict.items()}
results_df.columns = [qubo_to_key_dict[q] for q in results_df.columns[:-2]]+['energy', 'num_occurrences']
# sort by energy
results_df = results_df.sort_values(by=['energy'], ascending=True)

In [185]:
results_df.head(20)

,0_1,0_2,0_3,1_1,1_2,2_1,2_2,2_3,15_ho_term_1,16_ho_term_1,energy,num_occurrences
792,0,0,1,0,1,0,0,1,0,1,-3.0,1
119,0,0,1,1,0,0,1,0,0,0,-3.0,1
904,0,0,1,1,0,0,1,0,0,1,-3.0,1
286,1,0,0,0,1,0,0,1,1,0,-3.0,1
478,1,0,0,0,1,1,0,0,1,0,-3.0,1
39,0,0,1,0,1,1,0,0,0,0,-3.0,1
625,1,0,0,1,0,0,1,0,1,1,-3.0,1
264,0,0,1,1,0,0,0,1,1,0,-3.0,1
631,0,0,1,1,0,0,1,0,1,1,-3.0,1
796,0,1,0,0,1,0,0,1,0,1,-3.0,1


### Simulated annealing sampler

In [124]:
# encode the dataframe into a qubo object
bqm = dimod.BQM.from_qubo(qubo)
# SA solver
sampler_exact = dimod.SimulatedAnnealingSampler()
sampleset = sampler_exact.sample(bqm, num_reads=100, num_sweeps=500)

In [125]:
# save results in a dataframe
results_df = sampleset.to_pandas_dataframe()
# recover original indexes
qubo_to_key_dict = {v: k for k, v in key_to_qubo_dict.items()}
results_df.columns = [qubo_to_key_dict[q] for q in results_df.columns[:-2]]+['energy', 'num_occurrences']
# sort by energy
results_df = results_df.sort_values(by=['energy'], ascending=True)

In [126]:
results_df.head(10)

,0_0,0_1,10_13,11_0,11_1,12_0,12_1,13_0,13_1,14_13,...,11_0_11_1,40_ho_term,41_ho_term,13_0_13_1,4_0_4_1,44_ho_term,45_ho_term,46_ho_term,energy,num_occurrences
63,1,1,0,0,1,1,0,1,1,1,...,0,1,0,1,0,0,1,0,-20.0,1
86,1,1,0,0,1,0,0,1,1,1,...,0,1,1,1,0,1,1,0,-20.0,1
5,0,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,-20.0,1
60,1,0,0,0,1,0,0,1,1,1,...,0,1,1,1,0,1,1,1,-20.0,1
95,1,1,1,1,0,0,0,0,1,1,...,0,1,0,0,0,0,0,1,-19.0,1
6,1,1,1,0,1,0,0,1,1,1,...,0,1,1,1,0,0,0,0,-19.0,1
99,1,1,1,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,-19.0,1
58,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,1,0,0,-19.0,1
9,1,1,1,1,0,0,1,1,1,1,...,0,1,1,1,0,0,0,0,-19.0,1
32,1,1,1,0,0,0,0,0,1,1,...,0,1,0,0,0,1,0,1,-19.0,1


In [128]:
results_df.loc[86]

0_0                 1.0
0_1                 1.0
10_13               0.0
11_0                0.0
11_1                1.0
12_0                0.0
12_1                0.0
13_0                1.0
13_1                1.0
14_13               1.0
1_13                1.0
2_0                 1.0
2_1                 1.0
3_0                 1.0
3_1                 0.0
4_0                 1.0
4_1                 0.0
5_13                1.0
6_13                1.0
7_13                1.0
8_0                 1.0
8_1                 0.0
9_0                 1.0
9_1                 1.0
12_0_12_1           0.0
3_0_3_1             0.0
2_0_7_13            1.0
9_0_9_1             1.0
28_ho_term          1.0
29_ho_term          0.0
30_ho_term          0.0
31_ho_term          0.0
32_ho_term          1.0
33_ho_term          1.0
34_ho_term          1.0
35_ho_term          1.0
36_ho_term          1.0
37_ho_term          0.0
0_0_0_1             1.0
11_0_11_1           0.0
40_ho_term          1.0
41_ho_term      

In [258]:
results_df.groupby([k for k in key_to_qubo_dict.keys()]).agg({'energy': 'mean', 'num_occurrences': 'sum'}).sort_values(by=['energy'], ascending=True)

energy  \
0_1 0_2 0_3 10_13 11_1 11_2 11_3 12_1 12_2 12_3 13_1 13_2 13_3 14_13 1_13 2_1 2_2 2_3 3_1 3_2 3_3 4_2 5_13 6_13 7_13 8_1 8_2 8_3 9_1 9_2 9_3 58_ho_term_1 59_ho_term_1 61_ho_term_1           
0   1   0   0     0    0    1    1    0    0    1    0    0    1     1    1   0   0   0   0   0   1   1    1    1    1   0   0   0   1   0   0            0            1              -20.0   
                                                0    0    1    1     1    1   0   0   0   0   0   1   1    1    1    1   0   0   0   1   0   1            0            1              -20.0   
1   0   0   0     0    0    1    0    1    0    1    0    0    1     1    0   0   1   0   0   0   1   1    1    1    0   1   0   1   0   0   0            0            1              -20.0   
                                                0    0    1    1     1    1   0   0   0   0   0   1   1    1    1    0   0   1   1   0   0   1            0            1              -20.0   
0   1   0   1     0    0    1    0    0    0    1    0    0    1     0    0   0   1   1   0   0   1   1    1    1    1   0   0   0   1   0   1            1            1              -20.0   
...                                                                                                                                                                                     ...   
            0     1    0    0    0    0    1    1    0    0    1     1    0   0   1   0   0   0   1   1    1    1    1   0   0   0   1   0   0            0            1              -20.0   
    0   1   0     0    0    0    1    0    0    1    0    1    1     1    0   0   1   0   1   0   1   1    1    1    0   1   0   0   0   1   1            1            0              -19.8   
            1     0    1    0    0    0    0    0    0    1    1     1    0   0   1   1   0   0   1   1    1    1    1   0   0   0   0   1   0            1            1              -19.8   
                                                1    0    0    1     1    1   0   0   1   0   0   1   1    1    1    0   0   1   0   0   1   0            1            0              -19.8   
    1   0   0     0    0    0    0    0    1    0    0    1    1     1    1   0   0   0   0   0   1   1    1    1    0   0   1   1   0   0   0            1            1              -19.0   

                                                                                                                                                                                     num_occurrences  
0_1 0_2 0_3 10_13 11_1 11_2 11_3 12_1 12_2 12_3 13_1 13_2 13_3 14_13 1_13 2_1 2_2 2_3 3_1 3_2 3_3 4_2 5_13 6_13 7_13 8_1 8_2 8_3 9_1 9_2 9_3 58_ho_term_1 59_ho_term_1 61_ho_term_1                   
0   1   0   0     0    0    1    1    0    0    1    0    0    1     1    1   0   0   0   0   0   1   1    1    1    1   0   0   0   1   0   0            0            1                           1  
                                                0    0    1    1     1    1   0   0   0   0   0   1   1    1    1    1   0   0   0   1   0   1            0            1                           1  
1   0   0   0     0    0    1    0    1    0    1    0    0    1     1    0   0   1   0   0   0   1   1    1    1    0   1   0   1   0   0   0            0            1                           1  
                                                0    0    1    1     1    1   0   0   0   0   0   1   1    1    1    0   0   1   1   0   0   1            0            1                           1  
0   1   0   1     0    0    1    0    0    0    1    0    0    1     0    0   0   1   1   0   0   1   1    1    1    1   0   0   0   1   0   1            1            1                           1  
...                                                                                                                                                                                              ...  
            0     1    0    0    0    0    1    1    0    0    1     1    0   0   1   0   0   0   1   1  

In [168]:
results_df.loc[0]

0_0                 1.0
0_1                 1.0
1_0                 1.0
1_1                 1.0
2_0                 1.0
2_1                 1.0
8_ho_term           1.0
9_ho_term_1         1.0
9_ho_term_2         1.0
10_ho_term_1        1.0
10_ho_term_2        1.0
11_ho_term          1.0
12_ho_term_1        1.0
12_ho_term_2        1.0
13_ho_term          1.0
14_ho_term          1.0
15_ho_term_1        1.0
16_ho_term_1        1.0
17_ho_term          1.0
18_ho_term          1.0
21_ho_term          1.0
22_ho_term          1.0
energy            -33.6
num_occurrences     1.0
Name: 0, dtype: float64

In [ ]:
# TODO: - dense encoding DONE!
#       - wrap up encoding in a function DONE!
#       - take care of higher orders penalties (Boros DONE!) (Ishikawa DONE?!)
#       - 3 terms costraints DONE!
#       - look into bigger files DONE!

### Check Ishikawa

In [44]:
# read Boros results    
data_dir  = 'data/results/'
file_name = '8-Boros.txt'
with open(data_dir+file_name) as f:
    lines = f.readlines()

In [54]:
# save boros results in a dictionary ( only non-ancillary variables)
keys = []
values = []
avoid = [0, 1] + [len(lines)-i for i in range(1, 4)]
for i in range(len(lines)):
    if i in avoid: continue
    l = lines[i].split()
    keys.append(l[0])
    values.append(float(l[1]))

# retrieve variables
vars = [ key_to_qubo_dict[k] for k in keys]

# create boros dictionary
boros_results = {var: value for var, value in zip(vars, values)}

In [76]:
# dense encoding dictionary
dense_encoding_dict = {1: [0, 1], 2: [1, 0], 3: [1, 1], 13: [13]}

In [ ]:
for k, v in qubo_to_key_dict.items():
    qubo_to_key_dict[k] = v.split('_')

In [89]:
# produce ishikawa dict from boros results
ishikawa_results = {}
ishikawa_vars    = {}
i=0

for k in boros_results.keys():
    ishikawa_vars[i] = dense_encoding_dict[int(qubo_to_key_dict[k][1])]
    if np.isclose(boros_results[k], 1):
        if ishikawa_vars[i][0]<1:
            ishikawa_results[f'{qubo_to_key_dict[k][0]}_0' ] = 0
            ishikawa_results[f'{qubo_to_key_dict[k][0]}_1' ] = 1
        elif ishikawa_vars[i][0]>1:
            ishikawa_results[f'{qubo_to_key_dict[k][0]}_13' ] = 1
        elif ishikawa_vars[i][1]>1:
            ishikawa_results[f'{qubo_to_key_dict[k][0]}_0' ] = 1
            ishikawa_results[f'{qubo_to_key_dict[k][0]}_1' ] = 1
        else:
            ishikawa_results[f'{qubo_to_key_dict[k][0]}_0' ] = 1
            ishikawa_results[f'{qubo_to_key_dict[k][0]}_1' ] = 0
    elif ishikawa_vars[i][0]>1:
        ishikawa_results[f'{qubo_to_key_dict[k][0]}_13' ] = 0   
    i = i + 1

In [91]:
ishikawa_results

{'0_0': 1,
 '0_1': 0,
 '1_0': 0,
 '1_1': 1,
 '2_0': 1,
 '2_1': 0,
 '3_0': 1,
 '3_1': 0,
 '4_13': 1,
 '5_13': 0,
 '6_13': 1,
 '7_13': 1}

With Ishikawa we have 3 (15.spot standard encoding)  25 (8.spot dense encoding) additional variables, we consider all the possible instantiations

In [94]:
import itertools
inst = list(itertools.product([0, 1], repeat=25))

In [93]:
inst

[(0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 0, 1, 1),
 (0, 0, 0, 0, 0, 1, 0, 0),
 (0, 0, 0, 0, 0, 1, 0, 1),
 (0, 0, 0, 0, 0, 1, 1, 0),
 (0, 0, 0, 0, 0, 1, 1, 1),
 (0, 0, 0, 0, 1, 0, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, 1),
 (0, 0, 0, 0, 1, 0, 1, 0),
 (0, 0, 0, 0, 1, 0, 1, 1),
 (0, 0, 0, 0, 1, 1, 0, 0),
 (0, 0, 0, 0, 1, 1, 0, 1),
 (0, 0, 0, 0, 1, 1, 1, 0),
 (0, 0, 0, 0, 1, 1, 1, 1),
 (0, 0, 0, 1, 0, 0, 0, 0),
 (0, 0, 0, 1, 0, 0, 0, 1),
 (0, 0, 0, 1, 0, 0, 1, 0),
 (0, 0, 0, 1, 0, 0, 1, 1),
 (0, 0, 0, 1, 0, 1, 0, 0),
 (0, 0, 0, 1, 0, 1, 0, 1),
 (0, 0, 0, 1, 0, 1, 1, 0),
 (0, 0, 0, 1, 0, 1, 1, 1),
 (0, 0, 0, 1, 1, 0, 0, 0),
 (0, 0, 0, 1, 1, 0, 0, 1),
 (0, 0, 0, 1, 1, 0, 1, 0),
 (0, 0, 0, 1, 1, 0, 1, 1),
 (0, 0, 0, 1, 1, 1, 0, 0),
 (0, 0, 0, 1, 1, 1, 0, 1),
 (0, 0, 0, 1, 1, 1, 1, 0),
 (0, 0, 0, 1, 1, 1, 1, 1),
 (0, 0, 1, 0, 0, 0, 0, 0),
 (0, 0, 1, 0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0, 0, 1, 0),
 (0, 0, 1, 0, 0, 0, 1, 1),
 (0, 0, 1, 0, 0, 1, 0, 0),
 

In [126]:
# find the minimum energy
for comb in inst:
    new_results = boros_results.copy()
    for i in range(3):
        new_results[len(boros_results)+i] = comb[i]
    f = 0
    for i in range(len(new_results)):
        for j in range(len(new_results)):
            if (i, j) in qubo.keys():
                f = f + qubo[(i, j)]*new_results[i]*new_results[j]
    print(f, comb)

-17.8 (0, 0, 0)
-17.8 (0, 0, 1)
-17.8 (0, 1, 0)
-17.8 (0, 1, 1)
-20.0 (1, 0, 0)
-20.0 (1, 0, 1)
-20.0 (1, 1, 0)
-20.0 (1, 1, 1)
